In [10]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import distance
from scipy.linalg import sqrtm
import pandas as pd
np.random.seed(123)

In [11]:
# https://stackoverflow.com/questions/20295046/numpy-change-max-in-each-row-to-1-all-other-numbers-to-0
def BKM(X, gamma, c, n_steps):
    '''
    Implementation of Balanced K-Means (Algorithm 2).
    
    Attributes:
    ---
    X: ndarray
        Matrix (dxn), d is the number of features, n is the number of observations.
    gamma: float
        Regularization parameter.
    c: int
        Number of clusters.
    n_steps: int
        Number of iterations.
    
    Returns:
    ---
    F: ndarray
        Matrix (nxc) with predicted clusters.
    '''
    n = X.shape[1]
    A = np.random.normal(size=(n, c))
    F = np.zeros_like(A)
    F[np.arange(len(A)), A.argmax(1)] = 1
    
    for i in range(n_steps):
        H = X @ F @ np.linalg.inv(F.T @ F)
        F_new = F.copy()
        j = 0
        while (np.linalg.norm(F_new - F) == 0) & (j < 10):
            G = (n * gamma * np.identity(n) - gamma * np.ones(n).reshape(-1, 1) @ (np.ones(n).reshape(-1, 1).T)) @ F
            Q = G + X.T @ H - 1/2 * np.ones(n).reshape(-1, 1) @ (np.ones(X.shape[0]).reshape(-1, 1).T) @ (H * H)
            F_new = np.zeros_like(F_new)
            F_new[np.arange(len(Q)), Q.argmax(1)] = 1
            j += 1
        F = F_new.copy()
    
    return F

In [12]:
# https://stackoverflow.com/questions/48312205/find-the-k-nearest-neighbours-of-a-point-in-3d-space-with-python-numpy

def construct_similarity_matrix(X, W, k):
    '''
    Calculates the similarity matrix.
    
    Attributes:
    ---
    X: ndarray
        Matrix (dxn), d is the number of features, n is the number of observations.
    W: ndarray
        Matrix XF inv(F'F) where F is cluster indicator matrix
    k: int
        Number of neighbours.
    
    Returns:
    ---
    B: ndarray
        Similarity matrix (nxm).
    '''
    B = np.zeros((X.shape[1], W.shape[1]))
    D = distance.squareform(distance.pdist(X.T)) # Matrix of distances n x n 
    closest = np.argsort(D, axis=1)
    closest_inds = closest[:, 1 : k+2] # Indices of rows closest to row i, n x k
    
    for i in range(B.shape[0]):
        for j in range(B.shape[1]):
            if j in closest_inds[i, :-1]:
                B[i, j] = (((D[i, closest_inds[i, -1]]) ** 2) - np.linalg.norm(X[:, i] - W[:, j]) ** 2) /\
                (k * ((D[i, closest_inds[i, -1]]) ** 2) - np.sum((D[i, closest_inds[i, :-1]]) ** 2))
            else:
                B[i, j] = 0
    
    return B

In [23]:
def FNC(X, k, m, gamma, n_steps):
    '''
    Implementation of Fast Normalized Cut (Algorithm 3) from the article.
    
    Attributes:
    ---
    X: ndarray
        Matrix (dxn), d is the number of features, n is the number of observations.
    k: int
        Number of neighbours.
    m: int
        Number of clusters for BKM.
    gamma: float
        Regularization parameter.
    n_steps: int
        Number of iterations.
    
    Returns:
    ---
    Y: ndarray
        Matrix (nxc) with predicted clusters.
    '''
    n = X.shape[1]
    clusters = BKM(X, gamma, m, 100)
    
    W = X @ clusters @ np.linalg.inv(clusters.T @ clusters)
    B = construct_similarity_matrix(X, W, k)
    delta = np.diag(np.sum(B, axis = 0).tolist())
    PPT = B @ np.linalg.inv(delta) @ B.T
    
    A = np.random.normal(size=(n, m))
    Y = np.zeros_like(A)
    Y[np.arange(len(A)), A.argmax(1)] = 1
    
    for k in range(n_steps):
        G = PPT @ Y @ np.linalg.inv(sqrtm(Y.T @ Y))
        s = np.zeros_like(Y)
        for i in range(s.shape[0]):
            for l in range(s.shape[1]):
                s[i, l] = ((Y[:, l].T @ G[:, l] + G[i, l] * (1 - Y[i, l])) /\
                np.sqrt(Y[:, l].T @ Y[:, l] + (1 - Y[i, l]))) - ((Y[:, l].T @ G[:, l] - Y[i, l] * G[i, l])/\
                np.sqrt(Y[:, l].T @ Y[:, l] - Y[i, l]))
        Y_new = np.zeros_like(Y)
        Y_new[np.arange(len(s)), s.argmax(1)] = 1
        Y = Y_new.copy()
    
        if np.linalg.det(Y.T @ Y) == 0:
            return Y
    
    return Y

In [24]:
data = pd.read_csv('Data/isolet5.data')

In [25]:
X = data.iloc[:, :-1]
y = data.iloc[:, -1]

In [26]:
X.shape

(1558, 617)

In [27]:
pred = FNC(np.array(X.T), 20, 26, 0.1, 20)

In [31]:
np.argmax(pred, axis = 1)

(1558,)

In [32]:
np.sum(y == np.argmax(pred, axis = 1)) / 1558

0.03979460847240052